## Initialize the ckan environment and requests session

In [2]:
from os import path, environ
import requests
from dataflows import Flow, load
from datapackage_pipelines_ckanext.helpers import get_plugin_configuration

config = get_plugin_configuration('odata_org_il')
data_path = config['data_path']

CKAN_API_KEY = environ.get('CKAN_API_KEY')
CKAN_URL = environ.get('CKAN_URL')
assert CKAN_API_KEY and CKAN_URL
CKAN_AUTH_HEADERS = {'Authorization': CKAN_API_KEY}
session = requests.session()
session.headers.update(CKAN_AUTH_HEADERS)

DEBUG   :Starting new HTTPS connection (1): www.odata.org.il:443
DEBUG   :https://www.odata.org.il:443 "GET /pipelines/config/odata_org_il HTTP/1.1" 200 None


## Source entity data from foi site

In [18]:
from dataflows import Flow, load
import yaml

def process(rows):
    for row in rows:
        if int(row['nid']) == 446:
            print(yaml.dump(row, default_flow_style=False, allow_unicode=True))
            yield row

source_entity = Flow(
    load(data_path+'/new_foi_offices/datapackage.json'),
    process
).results()[0][0]

email: hofesh@taxes.mof.gov.il
fax: 03-6559305
form: '{"onlineurl": "~", "email": "hofesh@taxes.mof.gov.il ", "fax": "03-6559305\r\n",
  "letter": "\u05e8\u05d7'' \u05db\u05e0\u05e4\u05d9 \u05e0\u05e9\u05e8\u05d9\u05dd
  5, \u05ea.\u05d3. 1170, \u05d9\u05e8\u05d5\u05e9\u05dc\u05d9\u05dd 91010\r\n"}'
govxcontentsection: <p>רשות המיסים בישראל אחראית על גביית מיסים ישירים ועקיפים, הגברת
  האכיפה וההרתעה, מאבק בהון השחור וקידום הצמיחה והכלכלה בישראל.</p>
govxdescription: null
language: und
managername: שלמה אוחיון
mmdOfficesTypes: יחידת סמך
mmdOfficesTypes_tid: '58'
nid: '446'
officenamecode: '3047'
officetypecode: '3'
payments: '{"onlineurl": "~", "banktransfer": "\u05d1\u05e0\u05e7 \u05d4\u05d3\u05d5\u05d0\u05e8,
  \u05e1\u05e0\u05d9\u05e3 001, \u05d7\u05e9\u05d1\u05d5\u05df 50055", "postalbank":
  "\u05d1\u05e0\u05e7 \u05d4\u05d3\u05d5\u05d0\u05e8, \u05e1\u05e0\u05d9\u05e3 001,
  \u05d7\u05e9\u05d1\u05d5\u05df 50055", "treasury": "", "check": "", "phone": "",
  "cash": ""}'
receptionadd

## Matching ckan group from foi_groups_matching excel resource

In [53]:
from dataflows import Flow, load
import yaml

def process(rows):
    for i, row in enumerate(rows):
        if row['entity_id'] == f'foi-office-{source_entity["nid"]}':
            yield row

foi_group_matching_resource = Flow(load(data_path+'/foi_groups_matching/datapackage.json')).results()[0][0]
foi_group_matching_source_entity = [row for row in foi_group_matching_resource if row['entity_id'] == f'foi-office-{source_entity["nid"]}'][0]
print(yaml.dump(foi_group_matching_source_entity, default_flow_style=False, allow_unicode=True))

Column2: רשות המסים
Column3: 3c51d20a-e2c5-4266-a566-213de713682d
entity_id: foi-office-446
group_id: 44b29639-a5b1-41c9-a06b-e5ae3c17d86a
group_name: foi-office-446
matching ID: '1272'
title: 'יחידת סמך: רשות המסים בישראל'



## Load existing entities and find matching group

In [54]:
from dataflows import Flow, load
from load_existing_entities import get_existing_entities_resource, get_existing_entities_resource_descriptor
from collections import defaultdict

stats = defaultdict(int)
existing_entities_resource = Flow(load(({'resources': [get_existing_entities_resource_descriptor()]}, 
                                        [get_existing_entities_resource(stats)]))
                                 ).results()[0][0]
existing_entity = [row for row in existing_entities_resource if row['group_id'] == foi_group_matching_source_entity['Column3']][0]
print(yaml.dump(existing_entity, default_flow_style=False, allow_unicode=True))
print(f'num existing entities = {len(existing_entities)}')
print(dict(stats))

INFO    :Loading existing entities / groups, 500 results per page
INFO    :offset=0
DEBUG   :Starting new HTTPS connection (1): www.odata.org.il:443
DEBUG   :https://www.odata.org.il:443 "POST /api/3/action/group_list HTTP/1.1" 200 None
INFO    :offset=500
DEBUG   :Starting new HTTPS connection (1): www.odata.org.il:443
DEBUG   :https://www.odata.org.il:443 "POST /api/3/action/group_list HTTP/1.1" 200 None
INFO    :offset=1000
DEBUG   :Starting new HTTPS connection (1): www.odata.org.il:443
DEBUG   :https://www.odata.org.il:443 "POST /api/3/action/group_list HTTP/1.1" 200 None
INFO    :offset=1500
DEBUG   :Starting new HTTPS connection (1): www.odata.org.il:443
DEBUG   :https://www.odata.org.il:443 "POST /api/3/action/group_list HTTP/1.1" 200 None


entity_id: null
group:
  approval_status: approved
  created: '2017-09-03T12:24:13.629578'
  description: ''
  display_name: רשות המסים
  extras: []
  id: 3c51d20a-e2c5-4266-a566-213de713682d
  is_organization: false
  name: taxes
  num_followers: 0
  revision_id: 803d2f1e-c16d-4493-bd96-99a59726fb9b
  state: active
  title: רשות המסים
  type: group
group_id: 3c51d20a-e2c5-4266-a566-213de713682d
group_name: taxes
title: רשות המסים

num existing entities = 3
{'existing_groups_without_entity': 128, 'existing_groups_with_entity': 1154}


## Run dry run to update_foi_offices_entities manually only for this group

In [57]:
from update_foi_offices_entities import get_foi_offices_resource, get_existing_entities, get_foi_groups_matching
from collections import defaultdict
import yaml

stats = defaultdict(int)

existing_entities = {}
for row in get_existing_entities(existing_entities_resource, existing_entities, stats):
    pass

for row in get_foi_groups_matching(foi_group_matching_resource, existing_entities, stats):
    pass

for row in get_foi_offices_resource([source_entity], existing_entities, stats, True):
    print(yaml.dump(row, default_flow_style=False, allow_unicode=True))

INFO    :updating group id 3c51d20a-e2c5-4266-a566-213de713682d
DEBUG   :Starting new HTTPS connection (1): www.odata.org.il:443
DEBUG   :https://www.odata.org.il:443 "POST /api/3/action/group_patch HTTP/1.1" 200 None


email: hofesh@taxes.mof.gov.il
entity_id: foi-office-446
fax: 03-6559305
form: '{"onlineurl": "~", "email": "hofesh@taxes.mof.gov.il ", "fax": "03-6559305\r\n",
  "letter": "\u05e8\u05d7'' \u05db\u05e0\u05e4\u05d9 \u05e0\u05e9\u05e8\u05d9\u05dd
  5, \u05ea.\u05d3. 1170, \u05d9\u05e8\u05d5\u05e9\u05dc\u05d9\u05dd 91010\r\n"}'
govxcontentsection: <p>רשות המיסים בישראל אחראית על גביית מיסים ישירים ועקיפים, הגברת
  האכיפה וההרתעה, מאבק בהון השחור וקידום הצמיחה והכלכלה בישראל.</p>
govxdescription: null
language: und
managername: שלמה אוחיון
mmdOfficesTypes: יחידת סמך
mmdOfficesTypes_tid: '58'
nid: '446'
officenamecode: '3047'
officetypecode: '3'
payments: '{"onlineurl": "~", "banktransfer": "\u05d1\u05e0\u05e7 \u05d4\u05d3\u05d5\u05d0\u05e8,
  \u05e1\u05e0\u05d9\u05e3 001, \u05d7\u05e9\u05d1\u05d5\u05df 50055", "postalbank":
  "\u05d1\u05e0\u05e7 \u05d4\u05d3\u05d5\u05d0\u05e8, \u05e1\u05e0\u05d9\u05e3 001,
  \u05d7\u05e9\u05d1\u05d5\u05df 50055", "treasury": "", "check": "", "phone": "",
 

## Before updating - save the group datasets, otherwise they will be disconnected from group

In [ ]:
raise NotImplementedError()

## Do the update

In [ ]:
for row in get_foi_offices_resource([source_entity], existing_entities, stats, False):
    print(yaml.dump(row, default_flow_style=False, allow_unicode=True))

## Restore the datasets

In [58]:
from os import path, environ
import requests
from dataflows import Flow, load
from datapackage_pipelines_ckanext.helpers import get_plugin_configuration

def restore_group_datasets(row):
    group_id = row['group_id']
    if group_id == existing_entity['group_id']:
        for dataset_id in row['dataset_ids']:
            res = session.post('{}/api/3/action/member_create'.format(CKAN_URL),
                               json=dict(id=group_id,
                                         object=dataset_id,
                                         object_type='package',
                                         capacity='')).json()
            assert res and res['success']

Flow(
    load(path.join(data_path, 'dump_group_datasets/datapackage.json'), resources=['group_datasets']),
    restore_group_datasets
).process()


DEBUG   :Starting new HTTPS connection (1): www.odata.org.il:443
DEBUG   :https://www.odata.org.il:443 "POST /api/3/action/member_create HTTP/1.1" 200 None
DEBUG   :https://www.odata.org.il:443 "POST /api/3/action/member_create HTTP/1.1" 200 None
DEBUG   :https://www.odata.org.il:443 "POST /api/3/action/member_create HTTP/1.1" 200 None


(<datapackage.package.Package at 0x7fa6fab7add8>, {})